In [2]:
import os
os.chdir('..')

# Import necessary modules

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read dataset

In [4]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
species_map = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [6]:
df = pd.read_csv('example/data.csv')

In [7]:
df_X = df.drop(columns='species')
df_y = df['species'].map(species_map)

In [8]:
df_X

,sepal-length,sepal-width,petal-length,petal-width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [9]:
df_y

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: species, Length: 150, dtype: int64

In [10]:
x = df_X.values 
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_X = pd.DataFrame(x_scaled, columns=df_X.columns)

In [11]:
df_X.values

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667],
       [0.30555556, 0.79166667, 0.11864407, 0.125     ],
       [0.08333333, 0.58333333, 0.06779661, 0.08333333],
       [0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.02777778, 0.375     , 0.06779661, 0.04166667],
       [0.16666667, 0.45833333, 0.08474576, 0.        ],
       [0.30555556, 0.70833333, 0.08474576, 0.04166667],
       [0.13888889, 0.58333333, 0.10169492, 0.04166667],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.        , 0.41666667, 0.01694915, 0.        ],
       [0.41666667, 0.83333333, 0.03389831, 0.04166667],
       [0.38888889, 1.        , 0.08474576, 0.125     ],
       [0.30555556, 0.79166667, 0.05084746, 0.125     ],
       [0.22222222, 0.625     ,

# Create linguistic variables

In [12]:
from doggos.knowledge import LinguisticVariable, Domain

In [13]:
# features linguistic variables
sepal_length = LinguisticVariable('sepal-length', Domain(0, 1.001, 0.001))
sepal_width = LinguisticVariable('sepal-width', Domain(0, 1.001, 0.001))
petal_length = LinguisticVariable('petal-length', Domain(0, 1.001, 0.001))
petal_width = LinguisticVariable('petal-width', Domain(0, 1.001, 0.001))

In [14]:
# decision linguistic variable
species = LinguisticVariable('species', Domain(0, 1, 0.001))

# Define clauses

In [15]:
from doggos.knowledge import Clause
from doggos.fuzzy_sets import IntervalType2FuzzySet, Type1FuzzySet
from doggos.utils.membership_functions import triangular, trapezoidal, gaussian

In [16]:
# itype 2
clauses = [
    # clauses for petal length
    Clause(petal_length, 'small', IntervalType2FuzzySet(triangular(0, 0.25, 0.5, 0.8), triangular(0, 0.25, 0.5))),
    Clause(petal_length, 'medium', IntervalType2FuzzySet(triangular(0.25, 0.5, 0.75, 0.8), triangular(0.25, 0.5, 0.75))),
    Clause(petal_length, 'big', IntervalType2FuzzySet(triangular(0.5, 0.75, 1, 0.8), triangular(0.5, 0.75, 1))),
    # clauses for petal width
    Clause(petal_width, 'small', IntervalType2FuzzySet(gaussian(0.25, 0.1, 0.8), gaussian(0.25, 0.1))),
    Clause(petal_width, 'medium', IntervalType2FuzzySet(gaussian(0.5, 0.1, 0.8), gaussian(0.5, 0.1))),
    Clause(petal_width, 'big', IntervalType2FuzzySet(gaussian(0.75, 0.1, 0.8), gaussian(0.75, 0.1))),
    # claueses for sepal length
    Clause(sepal_length, 'small', IntervalType2FuzzySet(triangular(0, 0.25, 0.5, 0.8), triangular(0, 0.25, 0.5))),
    Clause(sepal_length, 'medium', IntervalType2FuzzySet(triangular(0.25, 0.5, 0.75, 0.8), triangular(0.25, 0.5, 0.75))),
    Clause(sepal_length, 'big', IntervalType2FuzzySet(triangular(0.5, 0.75, 1, 0.8), triangular(0.5, 0.75, 1))),
    # clauses for sepal width
    Clause(sepal_width, 'small', IntervalType2FuzzySet(gaussian(0.25, 0.1, 0.8), gaussian(0.25, 0.1))),
    Clause(sepal_width, 'medium', IntervalType2FuzzySet(gaussian(0.5, 0.1, 0.8), gaussian(0.5, 0.1))),
    Clause(sepal_width, 'big', IntervalType2FuzzySet(gaussian(0.75, 0.1, 0.8), gaussian(0.75, 0.1)))
]

# type 1
# clauses = [
#     # clauses for petal length
#     Clause(petal_length, 'small', Type1FuzzySet(triangular(0, 0.25, 0.5))),
#     Clause(petal_length, 'medium', Type1FuzzySet(triangular(0.25, 0.5, 0.75))),
#     Clause(petal_length, 'big', Type1FuzzySet(triangular(0.5, 0.75, 1))),
#     # clauses for petal width
#     Clause(petal_width, 'small', Type1FuzzySet(gaussian(0.25, 0.1))),
#     Clause(petal_width, 'medium', Type1FuzzySet(gaussian(0.5, 0.1))),
#     Clause(petal_width, 'big', Type1FuzzySet(gaussian(0.75, 0.1))),
#     # claueses for sepal length
#     Clause(sepal_length, 'small', Type1FuzzySet(triangular(0, 0.25, 0.5))),
#     Clause(sepal_length, 'medium', Type1FuzzySet(triangular(0.25, 0.5, 0.75))),
#     Clause(sepal_length, 'big', Type1FuzzySet(triangular(0.5, 0.75, 1))),
#     # clauses for sepal width
#     Clause(sepal_width, 'small', Type1FuzzySet(gaussian(0.25, 0.1))),
#     Clause(sepal_width, 'medium', Type1FuzzySet(gaussian(0.5, 0.1))),
#     Clause(sepal_width, 'big', Type1FuzzySet(gaussian(0.75, 0.1)))
# ]

In [17]:
for clause in clauses:
    print(clause)

Clause petal-length is small
Clause petal-length is medium
Clause petal-length is big
Clause petal-width is small
Clause petal-width is medium
Clause petal-width is big
Clause sepal-length is small
Clause sepal-length is medium
Clause sepal-length is big
Clause sepal-width is small
Clause sepal-width is medium
Clause sepal-width is big


# Define consequents

In [18]:
from doggos.knowledge.consequents import TakagiSugenoConsequent

In [19]:
parameters_1 = {sepal_length: 0.25, sepal_width: 0.25, petal_length: 0.5, petal_width: 0.5}
parameters_2 = {sepal_length: 0.15, sepal_width: 0.35, petal_length: 0.15, petal_width: 0.35}
parameters_3 = {sepal_length: 0.35, sepal_width: 0.15, petal_length: 0.35, petal_width: 0.15}
consequent_1 = TakagiSugenoConsequent(parameters_1, 0, species)
consequent_2 = TakagiSugenoConsequent(parameters_2, 1.5, species)
consequent_3 = TakagiSugenoConsequent(parameters_3, 0.7, species)

# Define terms

In [20]:
from doggos.knowledge import Term
from doggos.algebras import GodelAlgebra

In [21]:
terms = list()
algebra = GodelAlgebra()
for clause in clauses:
    terms.append(Term(algebra, clause))

# Define antecedents

In [22]:
# define random antecedents
# petal_length_small & petal_width_small
antecedent1 = terms[0] & terms[3]
# petal_width_small & sepal_length_big
antecedent2 = terms[3] & terms[8]
# sepal_width_medium | sepal-length_big | petal_width_big
antecedent3 = terms[10] | terms[8] | terms[5]

# Create rules

In [23]:
from doggos.knowledge import Rule

In [24]:
rules = [
    Rule(antecedent1, consequent_1),
    Rule(antecedent2, consequent_2),
    Rule(antecedent3, consequent_3)
]

# Fuzzify dataset

In [25]:
from doggos.knowledge import fuzzify

In [26]:
df_X_fuzzified = fuzzify(df_X, clauses)

In [27]:
measures = {sepal_length: df_X['sepal-length'].values, 
         sepal_width: df_X['sepal-width'].values, 
         petal_length: df_X['petal-length'].values, 
         petal_width: df_X['petal-width'].values}

# Create inference system

In [28]:
from doggos.inference import TakagiSugenoInferenceSystem
from doggos.inference.defuzzification_algorithms import takagi_sugeno_karnik_mendel, weighted_average

In [29]:
inference_system = TakagiSugenoInferenceSystem(rules)

In [30]:
result = inference_system.infer(weighted_average, df_X_fuzzified, measures)

TypeError: Axis must be specified when shapes of a and weights differ.

In [ ]:
result

# Make classification

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
theta1 = 0.38
theta2 = 0.72

In [ ]:
def classify(theta1, theta2):
    def _classify(x):
        if x < theta1:
            return 0
        elif theta1 <= x < theta2:
            return 1
        else:
            return 2
    return _classify

In [ ]:
classify_func = classify(theta1, theta2)
y_pred = list(map(lambda x: classify_func(x), result[species[0]]))

In [ ]:
accuracy = accuracy_score(y_pred, df_y.values)
print(f'Accuracy: {accuracy:.5f}')